In [ ]:
# !pip install -q git+https://github.com/seankmartin/simuran.git@main
# !pip install -q allensdk==2.14.1
# !pip install -q one-api==1.17.0
# !pip install -q ibllib==2.19.0

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import simuran as smr
from simuran.loaders.allen_loader import AllenVisualBehaviorLoader
from simuran.loaders.one_loader import OneAlyxLoader
import pandas as pd
from simuran.bridges.allen_vbn_bridge import (
    filter_good_units,
    get_brain_regions_to_structure_dict,
)
from simuran.plot.unit import plot_unit_properties

In [ ]:
try:
    from google.colab import drive
    drive.mount("/content/drive")
    data_directory = Path("/content/drive/MyDrive/NeuroScience/OpenData/")
except ModuleNotFoundError:
    data_directory = Path(r"D:\\")


In [ ]:
allen_name = "allen_npixels2"
ibl_name = "IBLdata"
ALLEN_MANIFEST = "visual-behavior-neuropixels_project_manifest_v0.4.0.json"
output_directory = data_directory / "OpenDataResults"

In [ ]:
def visualise_allen_units():
    allen_loader = AllenVisualBehaviorLoader(
        cache_directory=data_directory / allen_name, manifest=ALLEN_MANIFEST
    )
    all_units = allen_loader.get_all_units()
    plot_unit_properties(
        all_units,
        ["firing_rate"],
        [True],
        output_directory=output_directory / "allen" / "all_units",
        region_dict=get_brain_regions_to_structure_dict(),
        split_regions=False,
    )
    good_units = filter_good_units(all_units, sort_=False)
    plot_unit_properties(
        good_units,
        ["firing_rate"],
        [True],
        output_directory=output_directory / "allen" / "good_units",
        region_dict=get_brain_regions_to_structure_dict(),
        split_regions=False,
    )

In [ ]:
visualise_allen_units()

In [ ]:
def create_one_unit_table():
    one_loader = OneAlyxLoader.from_cache(data_directory / ibl_name)
    one_df = one_loader.get_sessions_table()
    one_recording_container = smr.RecordingContainer.from_table(one_df, one_loader)
    for i, r in enumerate(one_recording_container.load_iter()):
        header = (i == 0)
        for k, val in r.data.items():
            if not k.startswith("probe"):
                continue
            unit_table = val[1]
            eid = r.attrs.get("experiment_id")
            eid = r.attrs.get("session") if eid is None else eid
            unit_table["eid"] = eid 
            if header:
                columns = unit_table.columns
            else:
                unit_table = unit_table[columns]
                if unit_table.columns != columns:
                    raise ValueError(f"Invalid columns {unit_table.columns}")
            unit_table.to_csv(output_directory / "IBL" / "unit_table.csv", mode="a", index=False, header=header)

    return pd.read_csv(output_directory / "IBL" / "unit_table.csv", chunksize=50000)

In [ ]:
out_path = output_directory / "IBL" / "unit_table.csv"
if not out_path.exists():
    out_path.parent.mkdir(parents=True, exist_ok=True)
    df = create_one_unit_table()
    df.to_csv(out_path)